In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os

In [22]:
filepath = '/content/valka_s_mloky.txt'
with open(filepath) as f:
    capek_text = f.read()

In [23]:
print(capek_text[:200])

Karel Čapek
VÁLKA S MLOKY


Znění tohoto textu vychází z díla Válka s mloky tak, jak bylo vydáno v Československém spisovateli v roce 1981 (ČAPEK, Karel. Válka s mloky.  20. vyd. Praha : Československ


**Převod na tokeny**

In [24]:
"".join(sorted(set(capek_text.lower())))

'\t\n !%&()*,-./0123456789:;?[]abcdefghijklmnopqrstuvwxyz°áâçéíîóôöúüýčďęěňŕřšťůűž–‘’“”…'

In [25]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(capek_text)

In [26]:
max_id = len(tokenizer.word_index) # pocet ruznych znaku
dataset_size = tokenizer.document_count # celkovy pocet znaku

In [27]:
max_id

85

**Převod celého korpusu na tokeny**

In [28]:
[encoded] = np.array(tokenizer.texts_to_sequences([capek_text])) - 1
train_size = dataset_size * 90 // 100 # pro trenovani pouzijeme 90 % datasetu
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size]) # sekvence znak po znaku

In [29]:
encoded.shape

(410451,)

**Příprava datasetu pomocí knihovny tf.data**

window() nám převede dlouhou sekvenci (1M znaků) na okna o délce X (zde 100) znaků. shift=1 zajistí, že první instance jsou znaky 0-100, druhá instance 1-101 atd.

In [30]:
n_steps = 100
window_length = n_steps + 1 # target = input posunutý o 1 znak dopředu
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

window() nám dataset převede na dataset datasetů, což nemůžeme použít pro trénování. proto to splácneme zpátky na dataset sekvencí

In [31]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

GD funguje nejlépe, když jsou trénovací instance náhodně distribuované, proto dataset zamícháme a připravíme do dávek. Zároveň sekvenci o 101 znacích převedeme na prvních 100 znaků trénovacích a posledních 100 znaků jako target 

In [32]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

Protože různých znaků je málo, vystačíme si s one-hot encoding namísto word embeddings. Targety můžeme nechat jako celočíselné indexy. 

In [33]:
dataset = dataset.map(
    lambda X_batch, y_batch: (tf.one_hot(X_batch, depth=max_id), y_batch))

Prefetching předchystává dávky dat, aby byly nachystané už ve chvíli, kdy si je model vyžádá a nečekalo se na ně

In [34]:
dataset = dataset.prefetch(1)

**Zkontrolujeme výstup datasetu**

In [35]:
for X_batch, y_batch in dataset.take(1):
    print(X_batch.shape, y_batch.shape)

(32, 100, 85) (32, 100)


In [36]:
X_batch[0]

<tf.Tensor: shape=(100, 85), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [37]:
y_batch[0]

<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([ 8,  6,  0, 13,  3, 10,  2,  4, 19, 26,  0, 36, 20,  3, 12,  1,  0,
       30,  2, 13,  5, 22,  0, 14,  3,  4,  2, 26,  0, 20, 21,  0, 18,  1,
        0, 20,  2, 11,  4,  1, 15,  0,  9,  8, 11, 24,  6, 55,  0,  5,  1,
        5,  8, 28,  0, 20,  2,  4,  1, 10,  0, 20,  2, 18,  1,  0, 18,  6,
        3,  9, 15, 26,  0,  9, 13,  3, 16,  2,  6,  0, 20,  7,  2, 10,  0,
        7,  2,  0, 14,  1,  0, 30,  6, 15,  4, 15,  0,  1, 11,  0])>

**Připravíme a natrénujeme model**

Pokud nám má Dense vrstva vracet sekvenci, tak ji obalíme do TimeDistributed vrstvy, která se o to postará.

In [38]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [39]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, None, 128)         82560     
                                                                 
 time_distributed (TimeDistr  (None, None, 85)         10965     
 ibuted)                                                         
                                                                 
Total params: 93,525
Trainable params: 93,525
Non-trainable params: 0
_________________________________________________________________


In [40]:
history = model.fit(dataset, steps_per_epoch=train_size // batch_size,
                    epochs=5)

Epoch 1/5
11543/11543 [==============================] - 331s 28ms/step - loss: 2.1240
Epoch 2/5
11543/11543 [==============================] - 323s 28ms/step - loss: 1.9413
Epoch 3/5
11543/11543 [==============================] - 326s 28ms/step - loss: 1.8960
Epoch 4/5
11543/11543 [==============================] - 326s 28ms/step - loss: 1.8730
Epoch 5/5
11543/11543 [==============================] - 326s 28ms/step - loss: 1.8577


In [41]:
model.save('11_capek.keras')

In [42]:
model = keras.models.load_model('11_capek.keras')

**Generování znaků a sekvencí z natrénovaného modelu**

Protože jsme si upravovali ID tokenů, musíme si na to nachystat preprocessing i pro predikce

In [43]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

Protože jsme model učili predikovat sekvenci, vrátí nám metoda predict zase sekvenci. Znak, který by měl následovat po vstupních datech, je posledním znakem. 

In [44]:
X_new = preprocess(["Jak se má"])
X_new

<tf.Tensor: shape=(1, 9, 85), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.,

In [45]:
y_pred = np.argmax(model.predict(X_new), axis=-1)
y_pred

array([[ 2, 12,  1, 23,  2,  0, 14,  1, 10]])

In [46]:
tokenizer.sequences_to_texts(y_pred + 1)[0][-1] # ID zase o 1 zvedneme 

'm'

In [47]:
tokenizer.sequences_to_texts(y_pred + 1)[0]

'e k o b e   p o m'

Pomocí temperature můžeme stanovit, jak moc náhodná má být volba dalšího znaku z možných predikovaných

In [48]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [50]:
next_char("Jak se má", temperature=1)

'm'

In [51]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [52]:
print(complete_text("t", temperature=0.2))

to nebo podaří nebo čerti na mořský hlas do rozkazy


In [53]:
print(complete_text("t", temperature=1))

tan,” řešťal kníhavon robu.”
“nedává. halloc bandoe


In [54]:
print(complete_text("t", temperature=2))

távy sou-li níbic, s?evroščla – puk pisa! vám z pít
